In [1]:
import ROOT as rt

Welcome to JupyROOT 6.14/04


In [2]:
import argparse
import os
import sys

def setUnfoldBkgs(unfold_class, hfile_path, syst_name, isSys, nthSys, nTotSys, year, DYFake = False):

    if DYFake is False:
    
        unfold_class.subBkgs("Pt", hfile_path, "DYJetsToTauTau",       isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "DYJets10to50ToTauTau", isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "TTLL_powheg",          isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "WW_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "WZ_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "ZZ_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "WJets_MG",             isSys, syst_name, nTotSys, nthSys, "detector_level")

        unfold_class.subBkgs("Mass", hfile_path, "DYJetsToTauTau",       isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "DYJets10to50ToTauTau", isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "TTLL_powheg",          isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "WW_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "WZ_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "ZZ_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "WJets_MG",             isSys, syst_name, nTotSys, nthSys, "detector_level")
        
    else:
        unfold_class.subBkgs("Pt", hfile_path, "DYJets",       isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
        unfold_class.subBkgs("Mass", hfile_path, "DYJets",       isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
        if year == "2016":
            unfold_class.subBkgs("Pt", hfile_path, "DYJets10to50", isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
            unfold_class.subBkgs("Mass", hfile_path, "DYJets10to50", isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
        else:
            unfold_class.subBkgs("Pt", hfile_path, "DYJets10to50_MG", isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
            unfold_class.subBkgs("Mass", hfile_path, "DYJets10to50_MG", isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")

In [3]:
year    = "2016"
channel = "electron"
doSys   = True

phase_space_detector = 'fiducial_phase_pre_FSR_dRp1'
FSR_dR_detector      = 'dressed_dRp1'
phase_space_fsr      = 'full_phase_dRp1'
FSR_dR_fsr           = 'dRp1_pre_fsr'

# Set output directory
outputDirectory = 'output/'+year+'/' + channel + "/"
inputfhisttxtName = outputDirectory + "fhist.txt"

# Make output directory
if not os.path.exists( outputDirectory ):
    os.makedirs( outputDirectory )

# read text file including root file path for unfolding
fOutTxtCheck = open( inputfhisttxtName,'r')
unfoldInputList = {}

for line in fOutTxtCheck:
    modifiedLine = line.lstrip(' ').rstrip(' ').rstrip('\n')
    if modifiedLine.split()[1] == "matrix":
        unfoldInputList['matrix'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "matrix_pdferr":
        unfoldInputList['matrix_pdferr'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "matrix_closure":
        unfoldInputList['matrix_closure'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "fsr_matrix_closure":
        unfoldInputList['fsr_matrix_closure'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "matrix_alt":
        unfoldInputList['matrix_alt'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "fsr_matrix":
        unfoldInputList['fsr_matrix'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "fsr_matrix_alt":
        unfoldInputList['fsr_matrix_alt'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "fsr_photos_matrix":
        unfoldInputList['fsr_photos_matrix'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "fsr_pythia_matrix":
        unfoldInputList['fsr_pythia_matrix'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "gen_hist":
        unfoldInputList['gen_hist'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "gen_hist_closure":
        unfoldInputList['gen_hist_closure'] = modifiedLine.split()[2]

    if modifiedLine.split()[1] == "hist":
        unfoldInputList['hist'] = modifiedLine.split()[2]

print(unfoldInputList)

{'matrix': 'output/2016/electron/DY_v3.root', 'matrix_pdferr': 'output/2016/electron/DY_PDF.root', 'matrix_closure': 'output/2016/electron/ISR_response_closureTest.root', 'fsr_matrix_closure': 'output/2016/electron/ISR_response_FSR_closureTest.root', 'matrix_alt': 'output/2016/electron/DY_MG.root', 'fsr_matrix': 'output/2016/electron/DY_FSR.root', 'fsr_matrix_alt': 'output/2016/electron/DY_FSR_powheg.root', 'fsr_photos_matrix': 'output/2016/electron/DY_FSR_photos_v3.root', 'fsr_pythia_matrix': 'output/2016/electron/DY_FSR_pythia_v3.root', 'gen_hist': 'output/2016/electron/DY_Gen_Hists.root', 'gen_hist_closure': 'output/2016/electron/ISR_Gen_Hists_closureTest.root', 'hist': 'output/2016/electron/unfolding_input_v2.root'}


In [4]:
import pyScripts.unfoldUtil as unfoldutil
import pyScripts.drawUtil as drawutil

DetectorUnfold = 0;
FSRUnfold = 1;

bias = 1.0

# Create unfold class                                                                                       # regularization mode
unfoldClass = rt.ISRUnfold(channel, unfoldInputList['hist'], unfoldInputList['gen_hist'], False, int(year), int(0))
unfoldClass.setOutputBaseDir(outputDirectory)
unfoldClass.setBias(bias)

# Set response matrix
unfoldClass.SetNomTUnfoldDensity("Pt",  unfoldInputList['matrix'], phase_space_detector, FSR_dR_detector, unfoldInputList['matrix_closure'])
unfoldClass.SetNomTUnfoldDensity("Mass",unfoldInputList['matrix'], phase_space_detector, FSR_dR_detector, unfoldInputList['matrix_closure'])
unfoldClass.doClosureTest(DetectorUnfold, unfoldInputList['matrix_closure'], "detector_level")

# Set unfolding input histogram
unfoldClass.setInput("Pt",   unfoldInputList['hist'], False, "nominal", 0, bias, "detector_level")
unfoldClass.setInput("Mass", unfoldInputList['hist'], False, "nominal", 0, bias, "detector_level")
setUnfoldBkgs(unfoldClass, unfoldInputList['hist'],   "nominal", False, 0, -1, year)
#setUnfoldBkgs(unfoldClass, unfoldInputList['matrix'], "nominal", False, 0, -1, year, True)

i: 0 detector_level/hist_ptll/histo_DYJetsnominal
i: 0 detector_level_DY_Fake/hist_ptll/histo_DYJetsnominal
i: 1 detector_level_unfold_split_p5/hist_ptll/histo_DYJetsnominal
i: 1 detector_level_DY_Fake_unfold_split_p5/hist_ptll/histo_DYJetsnominal
i: 2 detector_level_unfold_split_p2/hist_ptll/histo_DYJetsnominal
i: 2 detector_level_DY_Fake_unfold_split_p2/hist_ptll/histo_DYJetsnominal
histo: detector_level/hist_ptll/histo_DYJetsToTauTaunominal
histo: detector_level/hist_ptll/histo_DYJets10to50ToTauTaunominal
histo: detector_level/hist_ptll/histo_TTLL_powhegnominal
histo: detector_level/hist_ptll/histo_WW_pythianominal
histo: detector_level/hist_ptll/histo_WZ_pythianominal
histo: detector_level/hist_ptll/histo_ZZ_pythianominal
histo: detector_level/hist_ptll/histo_WJets_MGnominal


Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 126 input bins and 70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 126 input bins and 70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 126 input bins and 70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnf

In [5]:
# Set systematic response matrix and input histograms
if channel == "electron": 
    sysDict = {"PU": 2, "trgSF": 2, "recoSF": 2, "IdSF": 2, "AlphaS": 2, "Scale": 6, "lepMom": 2, "Alt": 1, "Stat": 500} # "Stat"
if channel == "muon":
    sysDict = {"PU": 2, "trgSF": 2, "IsoSF": 2, "IdSF": 2, "AlphaS": 2, "Scale": 6, "Alt": 1, "Stat": 500}

if doSys == True:
    
    for sysName, nSys in sysDict.items():
        for nthSys in range(0,nSys):

            #print("sysName: " + sysName + " nthSys: " + str(nthSys) + " #####################################################")

            if sysName == "Alt":
                unfoldClass.setSysTUnfoldDensity("Pt",  unfoldInputList['matrix'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
                unfoldClass.setSysTUnfoldDensity("Mass",unfoldInputList['matrix'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
            elif sysName == "PDFerror":
                unfoldClass.setSysTUnfoldDensity("Pt",  unfoldInputList['matrix_pdferr'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
                unfoldClass.setSysTUnfoldDensity("Mass",unfoldInputList['matrix_pdferr'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
            else :
                # set systematic response matrix
                unfoldClass.setSysTUnfoldDensity("Pt",  unfoldInputList['matrix'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
                unfoldClass.setSysTUnfoldDensity("Mass",unfoldInputList['matrix'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)

            if sysName == "unfoldBias": bias = 0.95

            # set systematic input histograms
            unfoldClass.setInput("Pt",   unfoldInputList['hist'], True, sysName, nthSys, bias, "detector_level")
            unfoldClass.setInput("Mass", unfoldInputList['hist'], True, sysName, nthSys, bias, "detector_level")

            # set systematic background histograms
            if sysName is not "Stat":
                if sysName == "PDFerror":
                    setUnfoldBkgs(unfoldClass, unfoldInputList['hist'],          sysName, True, nthSys, nSys, year)
                    setUnfoldBkgs(unfoldClass, unfoldInputList['matrix_pdferr'], sysName, True, nthSys, nSys, year, True)
                else:
                    setUnfoldBkgs(unfoldClass, unfoldInputList['hist'],   sysName, True, nthSys, nSys, year)
                    setUnfoldBkgs(unfoldClass, unfoldInputList['matrix'], sysName, True, nthSys, nSys, year, True)

path: detector_level/hist_ptll/histo_DYJetsToTauTau_PUUp
path: detector_level/hist_ptll/histo_DYJets10to50ToTauTau_PUUp
path: detector_level/hist_ptll/histo_TTLL_powheg_PUUp
path: detector_level/hist_ptll/histo_WW_pythia_PUUp
path: detector_level/hist_ptll/histo_WZ_pythia_PUUp
path: detector_level/hist_ptll/histo_ZZ_pythia_PUUp
path: detector_level/hist_ptll/histo_WJets_MG_PUUp
path: detector_level_DY_Fake/hist_ptll/histo_DYJets_PUUp
path: detector_level_DY_Fake/hist_ptll/histo_DYJets10to50_PUUp
path: detector_level/hist_ptll/histo_DYJetsToTauTau_PUDown
path: detector_level/hist_ptll/histo_DYJets10to50ToTauTau_PUDown
path: detector_level/hist_ptll/histo_TTLL_powheg_PUDown
path: detector_level/hist_ptll/histo_WW_pythia_PUDown
path: detector_level/hist_ptll/histo_WZ_pythia_PUDown
path: detector_level/hist_ptll/histo_ZZ_pythia_PUDown
path: detector_level/hist_ptll/histo_WJets_MG_PUDown
path: detector_level_DY_Fake/hist_ptll/histo_DYJets_PUDown
path: detector_level_DY_Fake/hist_ptll/histo_

Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 126 input bins and 70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 112 input bins and 58 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #59
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 126 input bins and 70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnf

In [6]:
# Unfold for detector
unfoldClass.doISRUnfold(DetectorUnfold, doSys)

Info in <TUnfoldDensityV17::ScanTau>: logtau=-Infinity y=0.368475 X=2.872420 Y=6.564084
Info in <TUnfoldDensityV17::ScanTau>: logtau=-1.839389 y=0.431854 X=2.877946 Y=3.993704
Info in <TUnfoldDensityV17::ScanTay>: logtau=-2.339389 y=0.423233 X=2.875599 Y=4.954589
Info in <TUnfoldDensityV17::ScanTay>: logtau=-2.839389 y=0.354654 X=2.873350 Y=5.878132
Info in <TUnfoldDensityV17::ScanTay>: logtau=-3.339389 y=0.356634 X=2.872474 Y=6.429103
Info in <TUnfoldDensityV17::ScanTay>: logtau=-3.839389 y=0.367214 X=2.872421 Y=6.548847
Info in <TUnfoldDensityV17::ScanTay>: logtau=-4.339389 y=0.368348 X=2.872421 Y=6.562540
Info in <TUnfoldDensityV17::ScanTau>: logtau=-4.089389 y=0.368075 X=2.872421 Y=6.559217
Info in <TUnfoldDensityV17::ScanTau>: logtau=-3.589389 y=0.364542 X=2.872427 Y=6.517373
Info in <TUnfoldDensityV17::ScanTau>: logtau=-2.089389 y=0.430731 X=2.876874 Y=4.426365
Info in <TUnfoldDensityV17::ScanTau>: logtau=-3.089389 y=0.340543 X=2.872711 Y=6.224178
Info in <TUnfoldDensityV17::Scan

In [7]:
# Set QED FSR unfolding response matrix and input
unfoldClass.setNomFSRTUnfoldDensity("Pt",    unfoldInputList['fsr_matrix'], phase_space_fsr, FSR_dR_fsr, unfoldInputList['fsr_matrix_closure'])
unfoldClass.setNomFSRTUnfoldDensity("Mass",  unfoldInputList['fsr_matrix'], phase_space_fsr, FSR_dR_fsr, unfoldInputList['fsr_matrix_closure'])

unfoldClass.setFSRUnfoldInput(unfoldInputList['fsr_matrix'], False, "", -1, phase_space_fsr)

unfoldClass.doClosureTest(FSRUnfold, unfoldInputList['fsr_matrix_closure'], "fiducial_phase_dRp1")

Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 70 input bins and 70 output bins
Warning in <TUnfoldV17::TUnfold>: too few (ny=70) input bins for nx=70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnfoldV17::SetConstraint>: fConstraint=0
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 70 input bins and 70 output bins
Warning in <TUnfoldV17::TUnfold>: too few (ny=70) input bins for nx=70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnfoldV17::SetConstraint>: fConstraint=0
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 70 inp

In [8]:
if doSys == True:

    for sysName, nSys in sysDict.items():
        for nthSys in range(0,nSys):

            if sysName == "Alt":
                unfoldClass.setSysFSRTUnfoldDensity("Pt",   unfoldInputList['fsr_pythia_matrix'], sysName, nSys, nthSys, phase_space_fsr, FSR_dR_fsr)
                unfoldClass.setSysFSRTUnfoldDensity("Mass", unfoldInputList['fsr_pythia_matrix'], sysName, nSys, nthSys, phase_space_fsr, FSR_dR_fsr)

            else :
                unfoldClass.setSysFSRTUnfoldDensity("Pt",   unfoldInputList['fsr_matrix'], sysName, nSys, nthSys, phase_space_fsr, FSR_dR_fsr)
                unfoldClass.setSysFSRTUnfoldDensity("Mass", unfoldInputList['fsr_matrix'], sysName, nSys, nthSys, phase_space_fsr, FSR_dR_fsr)

            unfoldClass.setFSRUnfoldInput(unfoldInputList['fsr_matrix'], True, sysName, nthSys)

    # QED FSR ststematic
    sysDict["QED_FSR"] = 2

    # QED FSR ststematic
    unfoldClass.setSysFSRTUnfoldDensity("Pt",   unfoldInputList['fsr_photos_matrix'], "QED_FSR", 2, 0, phase_space_fsr, FSR_dR_fsr)
    unfoldClass.setSysFSRTUnfoldDensity("Mass", unfoldInputList['fsr_photos_matrix'], "QED_FSR", 2, 0, phase_space_fsr, FSR_dR_fsr)

    unfoldClass.setFSRUnfoldInput(unfoldInputList['fsr_matrix'], True, "QED_FSR", 0)

    unfoldClass.setSysFSRTUnfoldDensity("Pt",   unfoldInputList['fsr_pythia_matrix'], "QED_FSR", 2, 1, phase_space_fsr, FSR_dR_fsr)
    unfoldClass.setSysFSRTUnfoldDensity("Mass", unfoldInputList['fsr_pythia_matrix'], "QED_FSR", 2, 1, phase_space_fsr, FSR_dR_fsr)

    unfoldClass.setFSRUnfoldInput(unfoldInputList['fsr_matrix'], True, "QED_FSR", 1)


Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 70 input bins and 70 output bins
Warning in <TUnfoldV17::TUnfold>: too few (ny=70) input bins for nx=70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 58 input bins and 58 output bins
Warning in <TUnfoldV17::TUnfold>: too few (ny=58) input bins for nx=58 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #59
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 70 inp

In [9]:
# Unfold for QED FSR
unfoldClass.doISRUnfold(FSRUnfold, doSys)

Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 70 = input rank 70. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 58 = input rank 58. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 70 = input rank 70. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 70 = input rank 70. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 70 = input rank 70. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 70 = input rank 70. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 70 = input rank 70. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: number of parameters 70 = input rank 70. Problem is ill posed
Warning in <TUnfoldDensityV17::GetInputInverseEmatrix>: 

In [10]:
# Set nominal value and also systematic values
unfoldClass.setMeanPt(doSys, False, doSys)
unfoldClass.setMeanMass(doSys, False, doSys)

QED fsr check
mean1: 14.3723 mean2: 14.5502
QED fsr check
mean1: 14.3723 mean2: 14.5502
QED fsr check
mean1: 16.7482 mean2: 16.7333
QED fsr check
mean1: 16.7482 mean2: 16.7333
QED fsr check
mean1: 18.0924 mean2: 18.0824
QED fsr check
mean1: 18.0924 mean2: 18.0824
QED fsr check
mean1: 20.0027 mean2: 20.0512
QED fsr check
mean1: 20.0027 mean2: 20.0512
QED fsr check
mean1: 25.1507 mean2: 25.5064
QED fsr check
mean1: 25.1507 mean2: 25.5064
0 th mass bin... 
systematic detector unfold: AlphaS 0.380635
systematic detector unfold: Alt 0.270263
systematic detector unfold: IdSF 0.798532
systematic detector unfold: PU 0.285937
systematic detector unfold: Scale 0.820039
systematic detector unfold: Stat 0.25028
systematic detector unfold: lepMom 0.287186
systematic detector unfold: recoSF 0.498562
systematic detector unfold: trgSF 0.334266
systematic FSR unfold: AlphaS 0.395486
systematic FSR unfold: Alt 0.564248
systematic FSR unfold: IdSF 0.569254
systematic FSR unfold: PU 0.273731
systematic FS

Warning in <TFile::Append>: Replacing existing TH1: histMC_pt_temp (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: histMC_pt_fsr_temp (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: histMC_pt_temp (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: histMC_pt_fsr_temp (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: histMC_pt_temp (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: histMC_pt_fsr_temp (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: histMC_pt_temp (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: histMC_pt_fsr_temp (Potential memory leak).


In [11]:
# NOW DRAW PLOTS
dirClosurePlots = "ClosurePlots/"
if not os.path.exists( outputDirectory + dirClosurePlots ):
    os.makedirs( outputDirectory + dirClosurePlots )

dirUnfoldHists = "UnfoldHist/"
if not os.path.exists( outputDirectory + dirUnfoldHists ):
    os.makedirs( outputDirectory + dirUnfoldHists )

unfoldClass.drawNominalRecoPlots(outputDirectory + "Reco_" + channel , unfoldInputList['hist'], "Mass", 5)

for massBin in range(0,5):
    # Closure test
    unfoldClass.drawClosurePlots(DetectorUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "DetClosure_" + channel, "Pt", massBin)
    unfoldClass.drawClosurePlots(DetectorUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "DetClosure_" + channel, "Mass", massBin)
    unfoldClass.drawClosurePlots(FSRUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "FSRClosure_" + channel, "Pt", massBin)
    unfoldClass.drawClosurePlots(FSRUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "FSRClosure_" + channel, "Mass", massBin)

    #unfoldClass.drawNominalRecoPlots(outputDirectory + "Reco_" + channel , unfoldInputList['hist'], "Pt", massBin) # this line gives error please check.

    if not doSys:
        # Detector unfold
        unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel, "Pt", massBin)
        unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel, "Mass", massBin)

        # FSR unfold
        unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel, "Pt",   massBin, "", False, True)
        unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel, "Mass", massBin, "", False, True)

unfoldClass.drawClosurePlots(DetectorUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "DetClosure_" + channel, "Pt", 5)
unfoldClass.drawClosurePlots(FSRUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "FSRClosure_" + channel, "Pt", 5)


35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)


Info in <TCanvas::Print>: pdf file output/2016/electron/Reco_electron_input_5.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/ClosurePlots/DetClosure_electron_C0_Pt.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/ClosurePlots/DetClosure_electron_C0_Mass.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/ClosurePlots/FSRClosure_electron_C0_Pt.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/ClosurePlots/FSRClosure_electron_C0_Mass.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/ClosurePlots/DetClosure_electron_C1_Pt.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/ClosurePlots/DetClosure_electron_C1_Mass.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/ClosurePlots/FSRClosure_electron_C1_Pt.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/ClosurePlots/FSRClosure_ele

In [12]:
# Draw plots including systematic
if doSys == True:
    for sysName, nSys in sysDict.items():

        for massBin in range(0,5):

            unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel+sysName, "Pt", massBin, sysName, doSys)
            unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel+sysName, "Mass", massBin, sysName, doSys)
            unfoldClass.drawSysPlots(outputDirectory + "Sys_" + channel , massBin, sysName, True)
            unfoldClass.drawSysPlots(outputDirectory + "Sys_" + channel , massBin, sysName, False)

            unfoldClass.drawSysPlots(outputDirectory + "Sys_" + channel , massBin, "full", True)
            unfoldClass.drawSysPlots(outputDirectory + "Sys_" + channel , massBin, "full", False)

            unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ channel+"fullSys", "Pt", massBin, "full", doSys, True)
            unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ channel+"fullSys", "Mass", massBin, "full", doSys, True)

            if sysName is not "QED_FSR":
                # No QED_FSR systematic variabtion on detector unfolding
                unfoldClass.drawSysComparionPlots(outputDirectory + "Sys_" + channel , "Pt", massBin, sysName, True)
                unfoldClass.drawInputPlots(outputDirectory + channel + sysName, "Pt", massBin, sysName)

            unfoldClass.drawSysComparionPlots(outputDirectory + "Sys_" + channel , "Pt", massBin, sysName, False)

        if sysName is not "QED_FSR":
            unfoldClass.drawSysComparionPlots(outputDirectory + "Sys_" + channel , "Mass", massBin, sysName, True)
        unfoldClass.drawSysComparionPlots(outputDirectory + "Sys_" + channel , "Mass", massBin, sysName, False)

    unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ channel+"fullSys", "Mass", 5, "full", doSys, True)
    unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ channel+"fullSys", "Pt", 5, "full", doSys, True)

unfoldClass.drawISRMatrixInfo("Pt", outputDirectory, True)
unfoldClass.drawISRMatrixInfo("Pt", outputDirectory, False)

unfoldClass.drawISRMatrixInfo("Mass", outputDirectory, True)
unfoldClass.drawISRMatrixInfo("Mass", outputDirectory, False)

unfoldClass.drawSysSummaryPlots(outputDirectory + "ISRSysSummary_", False)
unfoldClass.drawISRresult(outputDirectory + "ISRfit_", True, "output/2016/" + channel + "/")

<ROOT.TCanvas object ("c_electron_2016") at 0x456eac60>

ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13 TeV)
delete done
ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13 TeV)
delete done
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13 TeV)
delete done
ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13 TeV)
delete done
35.9 fb^{-1} (13 TeV)
 ISRUnfold::drawInputPlots 
sysName: PU
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13 TeV)
delete done
ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13 TeV)
delete done
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13 TeV)
delete done
ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13 TeV)
delete done
35.9 fb^{-1} (13 TeV)
 ISRUnfold::drawInputPlots 
sysName: PU
35.9 fb^{-1} (13 TeV)
35.9 fb^{-1} (13 TeV)
ISRUnfold::drawUnfoldedHists called
35.9 fb^{-1} (13

Info in <TCanvas::Print>: pdf file output/2016/electron/UnfoldHist/Unfolded_electronPU_0_Pt_FSRUnfold.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/UnfoldHist/Unfolded_electronPU_0_Mass_FSRUnfold.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/Sys_electron_0_PU.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/Sys_electron_0_pre_fsr_PU.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/Sys_electron_0_full.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/Sys_electron_0_pre_fsr_full.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/Unfolded_electronfullSys_0_Pt_FSRUnfold.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/Unfolded_electronfullSys_0_Mass_FSRUnfold.pdf has been created
Info in <TCanvas::Print>: pdf file output/2016/electron/Sys_electronPt_distribution_0_PU.pdf has been created
Info i